# Sliding Window Decoding

Run both circuit-level and phenomenological sliding-window decoders.


In [1]:
import numpy as np
import stim

from quits.circuit import check_overlapping_CX
from quits import ErrorModel, CircuitBuildOptions
from quits.decoder import (
    sliding_window_bposd_circuit_mem,
    sliding_window_bposd_phenom_mem,
    sliding_window_bplsd_phenom_mem,
)
from quits.qldpc_code import BpcCode, HgpCode, QlpCode
from quits.simulation import get_stim_mem_result


### Decoder parameters
- `W`: sliding window width (number of rounds per window)
- `F`: overlap between consecutive windows
- `max_iter`: max BP iterations per window
- `osd_order`: OSD search depth for the BP-OSD decoder


In [2]:
# Common parameters
seed = 1
p = 1e-3
num_rounds = 15
num_trials = 100
W, F = 5, 3
max_iter, osd_order = 10, 1


In [3]:
# Build a small HGP code
h = np.loadtxt(
    "../parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt",
    dtype=int,
)
code = HgpCode(h, h)
report = code.verify_css_logicals()
print("verify_css_logicals", report)


verify_css_logicals {'css_condition': True, 'lz_commutes_with_X': True, 'lx_commutes_with_Z': True, 'rank_hz': 108, 'rank_hx': 108, 'rank_lz': 9, 'rank_lx': 9, 'k_expected': 9, 'lz_independent_mod_Z_stabilizers': True, 'lx_independent_mod_X_stabilizers': True, 'rank_hz_plus_lz': 117, 'rank_hx_plus_lx': 117, 'dim_ker_hz': 117, 'dim_ker_hx': 117, 'hx_plus_lx_spans_ker_hz': True, 'hz_plus_lz_spans_ker_hx': True, 'same_logicals_ZX_anticommute': True, 'different_logicals_ZX_commute': True, 'all_tests_passed': True}


In [4]:
# Generate memory experiment circuit.
# For this particular parity check matrix, seed=1 is the one that gives entangling depth 8.
# Other seeds may give entangling depth 12.
circuit = code.build_circuit(
    error_model=ErrorModel(p, p, p, p), 
    num_rounds=num_rounds, 
    basis="Z", 
    circuit_build_options=CircuitBuildOptions(), 
    seed=1
    )
check_overlapping_CX(circuit)
print("# layer of entangling gates: ", code.depth)
# print(circuit)

No overlapping CX gates found.
# layer of entangling gates:  8


In [5]:
# Simulate the circuit
detection_events, observable_flips = get_stim_mem_result(
    circuit, num_trials, seed=seed
)
print(detection_events.shape, observable_flips.shape)


(100, 1836) (100, 9)


### Circuit-level vs phenomenological decoding
- **Phenomenological decoding** uses an abstract noise model with data errors between rounds plus measurement-bit flips on syndrome outcomes. It does not model the full gate-by-gate extraction circuit.
- **Circuit-level decoding** models faults in the actual syndrome-extraction circuit (for example CNOT, reset, measurement, and idle errors), so a single fault can create correlated data and syndrome errors.
- Circuit-level decoding leads to lower LFRs, at the cost of longer decoding time. 
- See arXiv:2504.02673 for further detail. 

In [6]:
# Sliding window phenomenological decoding (BP-OSD)
depth = code.depth
eff_error_rate_per_fault = p * (depth + 3)

logical_pred = sliding_window_bposd_phenom_mem(
    detection_events,
    code.hz,
    code.lz,
    W,
    F,
    eff_error_rate_per_fault=eff_error_rate_per_fault,
    max_iter=max_iter,
    osd_order=osd_order,
    tqdm_on=True,
)

pL = np.mean((observable_flips - logical_pred).any(axis=1))
lfr = 1 - (1 - pL) ** (1 / num_rounds)
print({"p": p, "eff_error_rate_per_fault": float(eff_error_rate_per_fault), "pL": float(pL), "lfr": float(lfr)})


100%|██████████| 100/100 [00:02<00:00, 39.14it/s]

{'p': 0.001, 'eff_error_rate_per_fault': 0.011, 'pL': 0.04, 'lfr': 0.002717766469019667}


In [7]:
# Sliding window circuit-level decoding (BP-OSD)
logical_pred = sliding_window_bposd_circuit_mem(
    detection_events,
    circuit,
    code.hz,
    code.lz,
    W,
    F,
    max_iter=max_iter,
    osd_order=osd_order,
    tqdm_on=True,
)

pL = np.mean((observable_flips - logical_pred).any(axis=1))
lfr = 1 - (1 - pL) ** (1 / num_rounds)
print({"p": p, "pL": float(pL), "lfr": float(lfr)})


100%|██████████| 100/100 [00:19<00:00,  5.21it/s]

{'p': 0.001, 'pL': 0.01, 'lfr': 0.0006697979753554906}
